In [1]:
%cd /workspaces/Pointcept
from pointcept.engines.defaults import  default_setup
from pointcept.engines.defaults import (
    default_argument_parser,
    default_config_parser,
)
from pointcept.engines.train import TRAINERS
import torch
import numpy as np

/workspaces/Pointcept


/opt/conda/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
args = default_argument_parser().parse_args(
"""
--config-file /workspaces/Pointcept/configs/scannet/pipeline-sculpting-ptv3.py 
--num-gpus 0 
--options num_workers=0 num_worker_per_gpu=0 batch_size=1 
          weight=/workspaces/Pointcept/artifacts/spu8by4j-model_last:v80""".split()
)
cfg = default_config_parser(args.config_file, args.options)

In [3]:
cfg = default_setup(cfg)
#cfg.num_worker_per_gpu=0
trainer = TRAINERS.build(dict(type=cfg.train.type, cfg=cfg))

[2025-05-06 17:46:19,598 INFO train.py line 134 899930] => Loading config ...
[2025-05-06 17:46:19,599 INFO train.py line 136 899930] Save path: exp/default
[2025-05-06 17:46:19,909 INFO train.py line 137 899930] Config:
weight = '/workspaces/Pointcept/artifacts/spu8by4j-model_last:v80'
resume = False
evaluate = True
test_only = False
seed = 19929000
save_path = 'exp/default'
num_worker = 1
batch_size = 1
batch_size_val = None
batch_size_test = None
epoch = 800
eval_epoch = 100
clip_grad = None
sync_bn = False
enable_amp = True
amp_dtype = 'float16'
empty_cache = False
empty_cache_per_epoch = False
find_unused_parameters = False
mix_prob = 0.8
param_dicts = [dict(keyword='block', lr=0.0006)]
hooks = [
    dict(type='CheckpointLoaderAllowMismatch'),
    dict(type='IterationTimer', warmup_iter=2),
    dict(type='InformationWriter'),
    dict(type='SemSegEvaluator'),
    dict(type='CheckpointSaverWandb', save_freq=None)
]
train = dict(type='DefaultTrainer')
test = dict(type='SemSegPredict

In [4]:
train_loader = trainer.train_loader
i, b = next(enumerate(train_loader))

b

{'coord': tensor([[-1.1505,  0.2562,  1.2328],
         [-1.1505,  0.2562,  1.2528],
         [-1.1226,  0.2514,  1.2328],
         ...,
         [ 1.0038,  1.2957,  2.4845],
         [ 0.8311,  1.4404,  2.6325],
         [ 1.0625, -0.5596, -0.0807]]),
 'grid_coord': tensor([[221,  70,  70],
         [221,  70,  57],
         [221,  70,  58],
         ...,
         [ 32, 163, 127],
         [ 32, 163, 118],
         [ 32, 163, 119]]),
 'segment': tensor([0, 0, 0,  ..., 0, 1, 0]),
 'offset': tensor([120000]),
 'feat': tensor([[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         ...,
         [0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]])}

In [5]:
b.keys()

dict_keys(['coord', 'grid_coord', 'segment', 'offset', 'feat'])

In [ ]:
from pointcept.utils.events import EventStorage, ExceptionWriter
import pointcept.utils.comm as comm

with EventStorage() as trainer.storage, ExceptionWriter():
    # => before train
    trainer.before_train()
    trainer.logger.info(">>>>>>>>>>>>>>>> Start Training >>>>>>>>>>>>>>>>")
    for trainer.epoch in range(1):
        # => before epoch
        trainer.model.train()
        trainer.data_iterator = enumerate(trainer.train_loader)
        trainer.before_epoch()
        # => run_epoch
        for (
            trainer.comm_info["iter"],
            trainer.comm_info["input_dict"],
        ) in trainer.data_iterator:
            # => before_step
            trainer.before_step()
            # => run_step
            trainer.run_step()
            # => after_step
            trainer.after_step()
        # => after epoch
        trainer.after_epoch()
    # => after train
    trainer.after_train()

KeyError: 'input_dict'